In [ ]:
import os
from dotenv import load_dotenv

import textwrap 
import numpy as np
import pandas as pd

import google.generativeai as genai

from IPython.display import Markdown

In [ ]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

Embedding

In [ ]:
title = "The next geenration of AI for developers and Google Workspace"
sample_text = ("Title: The next generation of AI for developers and Google Workspace",)
model = 'models/embedding-001'
embedding = genai.embed_content(model=model,content=sample_text,task_type="retrieval_document",
title=title)
print(embedding)
print(len(embedding['embedding'][0]))


Extraction of text from PDF file

In [ ]:
from pdfminer.high_level import extract_text

text = extract_text(r"C:\Users\aanan\Desktop\My Folder\Major Project_REPORT.pdf")
# split after every 5 sentences
sentences = text.split(".")
sentences = [" ".join(sentences[i: i + 10]).replace('\n',' ').strip() for i in range(0,len(sentences), 10)]
for i in sentences:
    print(i)


In [ ]:
documents = []
for idx, i in enumerate(sentences):
    documents.append({
        'title': f"Document {idx}",
        'content': i
    })
documents 

In [ ]:
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

In [ ]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(title, text):
    return genai.embed_content(
        model=model,
        content=text,
        task_type="retrieval_document",
        title=title
    )["embedding"]

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'],row['Text']), axis=1)
df

question & its embedding

In [ ]:
query = "The development server automatically"
model = 'models/embedding-001'

request = genai.embed_content(
    model=model,
    content=query,
    task_type="retrieval_query"
)
print(request)

In [ ]:
def find_best_passage(query, dataframe):
    """
    Compute the distances between the query and each document in the dataframe using the dot product
    """
    
    query_embedding = genai.embed_content(model=model,
                                          content=query,
                                          task_type="retrieval_query")
    dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
    idx = np.argmax(dot_products)
    return dataframe.iloc[idx]['Text']  # Return text from index with max value

In [ ]:
passage = find_best_passage(query, df)
passage

In [ ]:
def make_promp(query, relevant_passage):
    escaped = relevant_passage.replace("'","").replace('"',"").replace("\n"," ") # escaped means to give data after cleaning
    prompt = textwrap.dedent("""You are a helpful and informative bot that answers \
    questions using text from the reference passage included below. \
    Be sure to respond in a complete sentence, being comprehensive, \
    including all relevant background information. \
    However, you are talking to a non-technical audience, \
    so be sure to break down complicated concepts and \
    strike a friendly and conversational tone. \
    If the passage is irrelevant to the answer, you may ignore it.
    QUESTION : '{query}'
    PASSAGE : '{relevant_passage}'
    ANSWER :
    """).format(query=query, relevant_passage=escaped)
    
    return prompt

In [ ]:
prompt = make_promp(query , passage)
print(prompt)

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-flash-latest')
answer = model.generate_content(prompt)
Markdown(answer.text)